### Library

In [1]:
import os
from dotenv import load_dotenv

from binance.client import Client

import pandas as pd
import numpy as np
from datetime import datetime
import ta

from sklearn.neighbors import KNeighborsClassifier

from backtesting import Backtest, Strategy

### TA Calculation Functions

#### SMA

In [2]:
def sma(df, window):
    sma = ta.trend.SMAIndicator(pd.Series(df), window=window).sma_indicator()
    return sma

#### RSI

In [3]:
def rsi(df, window=14):
    rsi = ta.momentum.RSIIndicator(pd.Series(df), window=window).rsi()
    return rsi

#### EMA

In [4]:
def ema(df, period=200):
    ema = ta.trend.EMAIndicator(pd.Series(df), window=window).ema_indicator()
    return ema

#### MACD

In [5]:
def macd(df):
    macd = ta.trend.MACD(pd.Series(df)).macd()
    return macd

#### Bollinger Bands

In [6]:
def signal_h(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_hband()
def signal_l(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_lband()

### other handy functions

In [7]:
def get_X(data):
    return data.filter(like='X').values

In [8]:
def get_y(data):
    y = data['Close'].pct_change(48).shift(-48)
    y[y.between(-.004, .004)] = 0             # Devalue returns smaller than 0.4%
    y[y > 0] = 1
    y[y < 0] = -1
    return y

In [18]:
def get_clean_Xy(data):
    X = get_X(data)
    y = get_y(data).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y

### Strategy Class

#### ML Strategy

In [22]:
class MLstr(Strategy):
    price_delta = 0.004
    train_split = .7 # 70% training 30% testing

    def init(self):        
        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier(7)

        N_train = int(self.data.df.shape[0] * self.train_split)

        # Train the classifier in advance on the first N examples
        df = self.data.df.iloc[:N_train]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')

    def next(self):
        # Skip the training, in-sample data
        N_train = int(self.data.df.shape[0] * self.train_split)

        if len(self.data) < N_train:
            return

        # Proceed only with out-of-sample data. Prepare some variables
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]

        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        upper, lower = close[-1] * (1 + np.r_[1, -1] * self.price_delta)
 
        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        # Additionally, set aggressive stop-loss on trades that have been open 
        # for more than two days
        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta('2 days'):
                if trade.is_long:
                    trade.sl = max(trade.sl, low)
                else:
                    trade.sl = min(trade.sl, high)


### ML Model


In [11]:
def featureGeneration(data):
    close = data.Close

    sma10 = sma(data.Close, 10)
    sma20 = sma(data.Close, 20)
    sma50 = sma(data.Close, 50)
    sma100 = sma(data.Close, 100)
    upper = signal_h(data.Close)
    lower = signal_l(data.Close)

    # Design matrix / independent features:

    # Price-derived features
    data['X_SMA10'] = (close - sma10) / close
    data['X_SMA20'] = (close - sma20) / close
    data['X_SMA50'] = (close - sma50) / close
    data['X_SMA100'] = (close - sma100) / close

    data['X_DELTA_SMA10'] = (sma10 - sma20) / close
    data['X_DELTA_SMA20'] = (sma20 - sma50) / close
    data['X_DELTA_SMA50'] = (sma50 - sma100) / close

    # Indicator features
    data['X_MOM'] = data.Close.pct_change(periods=2)
    data['X_BB_upper'] = (upper - close) / close
    data['X_BB_lower'] = (lower - close) / close
    data['X_BB_width'] = (upper - lower) / close

    # Some datetime features for good measure
    data['X_day'] = data.index.dayofweek
    data['X_hour'] = data.index.hour

    data = data.dropna().astype(float)
    return data

## Main

### Binance API

In [12]:
load_dotenv()
API_KEY    = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

client = Client(API_KEY, API_SECRET)

### Parameters

### Fetch Historical Data

In [13]:
def fetchData(client, start_time, end_time, symbol, interval):
    kline = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time, end_str=end_time)

    columns = ['index','Open', 'High', 'Low', 'Close', 'Volume']

    data = pd.DataFrame(kline)
    data = data.iloc[:, :6]
    data.columns  = columns
    data['index'] = pd.to_datetime(data['index'], unit='ms')
    data.set_index('index', inplace=True)
    data = data.astype(float)

    return data

In [20]:
symbol = 'BTCUSDT'
interval = '1h'

start_time = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_time    = int(datetime(2023,12,31,0,0).timestamp() * 1000)

data = fetchData(client=client, start_time=start_time, end_time=end_time, symbol=symbol, interval=interval)
data = featureGeneration(data)

### Backtesting

#### Backtesting

#### Graph

## For Temp Ad Hoc Testing

In [23]:
bt = Backtest(data, MLstr, cash = 1000000, commission = .0007, margin = .05)
bt.run()
bt.plot()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:128: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  indicators = [_Indicator(i.df.resample(freq, label='right').mean()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:128: FutureWarning: 'H' is depr

Row(id='1543', ...)

In [ ]:
np.r_[1, -1]

## Reference

Backtesting User Manual

https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html